In [1]:
import os
import bs4
from bs4 import BeautifulSoup as soup
import pandas as pd
import spacy
import re
import en_core_web_sm
import keras
import tensorflow
from pathlib import Path
from pathlib import PureWindowsPath

Using TensorFlow backend.


In [2]:
# nlp = spacy.load('en')

In [3]:
def extract_html(input_html):
    with open(input_html) as inf:
        txt = inf.read()
    soup_out = soup(txt, "html.parser")
    print(input_html)
    return soup_out

In [4]:
def extract_firm(input_soup):
    temp = input_soup.find(id='article_participants')
    for element in temp.findAll('a'):
        if element.get('title') != None :
            return(str(element.get('title')))   
    return None

In [5]:
def extract_ticker(input_soup):
    temp = input_soup.find(id='article_participants')
    for element in temp.findAll('a'):
        if element.get('title') != None :
            return(str(element.contents[0].strip()))   
    return None

In [6]:
def extract_time(input_soup):
    temp = input_soup.find(id='article_participants')
    try:
        for element in temp.findAll('p'):
            if (pd.notnull(element.text) and element.text!=''):
                if( re.search('call',element.contents[0],flags=re.IGNORECASE)):
                    call_title = element.contents[0].split()
                    return str(call_title[0]),str(call_title[1])
        return None,None
    except:
        return None,None
    
def extract_quarter(input_soup):
    temp = input_soup.find(id='article_participants')
    try:
        for element in temp.findAll('p'):
            if (pd.notnull(element.text) and element.text!=''):
                if( re.search('call',element.contents[0],flags=re.IGNORECASE)):
                    call_title = element.contents[0].split()
                    return str(call_title[0])
        return None
    except:
        return None
    
def extract_year(input_soup):
    temp = input_soup.find(id='article_participants')
    try:
        for element in temp.findAll('p'):
            if (pd.notnull(element.text) and element.text!=''):
                if( re.search('call',element.contents[0],flags=re.IGNORECASE)):
                    call_title = element.contents[0].split()
                    return str(call_title[1])
        return None
    except:
        return None

In [7]:
def test_analyst(input_soup):
    temp = input_soup.find(id='article_participants')
    for element in temp.findAll('strong'):
        if(re.search('analyst',element.contents[0],flags=re.IGNORECASE)):
            return(True)
    return(None)


In [8]:
def extract_analysts(input_soup):
    temp = input_soup.find(id='article_participants')
    test_string = []
    print_flag = False
    for element in temp.find_all('p'):
        if(re.search('analyst',element.text,flags=re.IGNORECASE)):
            #print(element.text.strip(),'###')
            element = element.find_next()
            while  (1):
                element = element.find_next()
            #print(element.contents[0].strip())
            #print(element.text.strip())
                if element.find('strong') or \
                element.find('div'):
            #print('FOUND')
                    break
                if (pd.notnull(element.text) and element.text!=''):
                    test_string.append(element.contents[0].strip())    
            break
    return test_string

In [9]:
def extract_executives(input_soup):
    test_string = []
    temp = input_soup.find(id='article_participants')
    # print(soup_out)
    print_flag = False
    for element in temp.find_all('p'):
        if(re.search('executive',element.text,flags=re.IGNORECASE)):
            #print(element.text.strip(),'###')
            element = element.find_next()
            while  (1):
                element = element.find_next()
                # print(element.contents[0])
                # print(element.text.strip())
                if element.find('strong') or \
                element.find('div'):            
                #print('FOUND')
                    break
                test_string.append(element.text.strip())
        
            break
    return test_string

In [10]:
def is_analyst_present(analyst_list):
    if analyst_list: 
        return True
    else: 
        return False

In [11]:
def extract_name (input_string,delim='-'):
    if not pd.isnull(input_string):
        name_search = re.search('(^[^-]*)-', input_string, re.IGNORECASE)
        try:
            return name_search.group(1).strip()
        except:
            return None        
    else:
        return None
def extract_title (input_string,delim='-'):
    if not pd.isnull(input_string):
        title_search = re.search('^[^-]*-(.*)', input_string, re.IGNORECASE)
        try:
            return title_search.group(1).strip()
        except:
            return None           
        else:
            return None
    else:
        return None

In [12]:
def extract_content(input_soup,current_name):
#     current_name = 'Panna Sharma'
    if not (pd.isnull(current_name) ):
        temp = input_soup.find(id='article_content')
        test_string = []
        # print(temp)
        for element in temp.find_all('strong'):
            if (re.search(current_name.strip().lower(),element.text,flags=re.IGNORECASE)):
                while  (1):
                    element = element.find_next()
        #           print(element.text.strip())
                    if element.find('strong') or \
                    element.find('div'):
                        break
                    test_string.append(element.text.strip())    
        return test_string
    else:
        return None

In [13]:
def extract_qanda(input_soup,current_name):
#     current_name = 'Panna Sharma'
    if not (pd.isnull(current_name) ):
        temp = input_soup.find(id='article_qanda')
        test_string = []
        # print(temp)
        if temp.find_all('span'):
            for element in temp.find_all('span'):
                if (re.search(current_name.strip().lower(),element.text,flags=re.IGNORECASE)):
                    while  (1):
                        element = element.find_next()
            #           print(element.text.strip())
                        if element.find('strong') or \
                        element.find('div') or \
                        element.find('span'):
                            break
                        test_string.append(element.text.strip())    
        else:                
            for element in temp.find_all('strong'):
                if (re.search(current_name.strip().lower(),element.text,flags=re.IGNORECASE)):
                    while  (1):
                        element = element.find_next()
            #           print(element.text.strip())
                        if element.find('strong') or \
                        element.find('div') or \
                        element.find('span'):
                            break
                        test_string.append(element.text.strip())      
            
        return test_string
    else:
        return None

In [14]:
nlp = en_core_web_sm.load()

def basic_wc(input_text):
    if not (input_text==[]):
#         decoded_txt=input_text.decode('utf-8')
        sent =  " ".join(str(x) for x in input_text)
        sent = re.sub(pattern='[^a-zA-Z0-9 -]',repl='',string=sent)            
        doc = nlp(sent)
        return doc.__len__()
    else:
        return 0


In [15]:
def extract_chunks(input_text):
    sent =  " ".join(str(x) for x in input_text)
    #     doc = nlp(input_text)
    doc = nlp(sent)
    spans = list(doc.ents)  #+ list(doc.noun_chunks)
    out_list = []
    for span in spans:
        span.merge()
        out_list.append(span)
    return out_list

In [16]:
def extract_name_pair(name, input_text):
    out_list = []
    if not (input_text==[]):
#         s_out = extract_chunks(input_text)  
        sent =  " ".join(str(x) for x in input_text)
        #     doc = nlp(input_text)
        doc = nlp(sent)        
#         doc = nlp(input_text)
        spans = list(doc.ents)  #+ list(doc.noun_chunks)

        for item in spans:
            try:
                item = re.sub(pattern='[^a-zA-Z0-9 -]',repl='',string=item.string)            
                if (re.search(item,name,flags=re.IGNORECASE)):
                    out_list.append(item)

            except:
                pass
            
    return out_list
#     else:
#         return None

In [17]:
collection = "html"

# n_test=50
file_list = os.listdir(collection)
# for i in range(len(file_list)):
# for i in range(n_test):
#     print(file_list[i])
print(len(file_list))

38490


In [18]:
# collection = "html"
# data_folder = Path(collection)
output_folder = PureWindowsPath('C:/Users/T.Santos18/Documents/workspace/textparsing/exec')
input_folder = PureWindowsPath('C:/Users/T.Santos18/Documents/workspace/textparsing/html')

n_test=10
file_list = os.listdir(input_folder)
out_list =[]
# for i in range(len(file_list)):
print('FILE\tANALYST_Flag\tFIRM\tQuarter\tYear') 
print(len(file_list))
for i in range(n_test):
    with open(input_folder / file_list[i]) as inf:
        if(re.search('html',file_list[i],flags=re.IGNORECASE)):
            out_list.append(file_list[i])
            print(file_list[i])

FILE	ANALYST_Flag	FIRM	Quarter	Year
91979
1.html
10.html
100.html
1000.html
10000.html
10001.html
10002.html
10003.html
10004.html


In [27]:
out_pd = pd.DataFrame(out_list)
out_pd.columns=['file']
out_pd['text'] = out_pd.apply(lambda x: extract_html('html/'+x['file']), axis=1)
out_pd['quarter'] = out_pd.apply(lambda x: extract_quarter(x['text']), axis=1)
out_pd['year'] = out_pd.apply(lambda x: extract_year(x['text']), axis=1)
# out_pd['quarter_year'] = out_pd.apply(lambda x: extract_q(x['text']), axis=1)
out_pd['executive_firm'] = out_pd.apply(lambda x: extract_firm(x['text']), axis=1)
out_pd['analyst_present'] = out_pd.apply(lambda x: test_analyst(x['text']), axis=1)
out_pd['ticker'] = out_pd.apply(lambda x: extract_ticker(x['text']), axis=1)

out_pd['analysts'] = out_pd.apply(lambda x: extract_analysts(x['text']), axis=1)
out_pd['execs'] = out_pd.apply(lambda x: extract_executives(x['text']), axis=1)

s = out_pd.execs.apply(pd.Series)\
              .stack()\
              .reset_index(level=1, drop=True)\
              .to_frame('executive')
out_executives = out_pd.join(s)                      


out_executives['exec_name'] = out_executives.apply(lambda x: extract_name(x['executive']), axis=1)
out_executives['exec_title'] = out_executives.apply(lambda x: extract_title(x['executive']), axis=1)

out_executives['executive_content'] = out_executives.apply(lambda x: \
                                         extract_content(x['text'],x['exec_name']), axis=1)
out_executives['executive_qanda'] = out_executives.apply(lambda x: \
                                         extract_qanda(x['text'],x['exec_name']), axis=1)

out_executives['exec_qna_count'] = out_executives.apply(lambda x: \
                                         basic_wc(x['executive_qanda']), axis=1)

out_executives['exec_content_count'] = out_executives.apply(lambda x: \
                                         basic_wc(x['executive_content']), axis=1)
out_executives['exec_total_count'] = out_executives.exec_qna_count + out_executives.exec_content_count

out_executives=out_executives.reset_index(level=0, drop=True)
# out_analyst
s = out_executives.execs.apply(pd.Series)\
              .stack()\
              .reset_index(level=1, drop=True)\
              .to_frame('mention_name')
out_executives = out_executives.join(s)          
out_executives['mention_name'] = out_executives.apply(lambda x: extract_name(x['mention_name']), axis=1)

out_executives['exec_mention_qna'] = out_executives.apply(lambda x: extract_name_pair(x['mention_name'],x['executive_qanda']), axis=1)
out_executives['exec_mention_transcript'] = out_executives.apply(lambda x: extract_name_pair(x['mention_name'],x['executive_content']), axis=1)
out_executives['exec_count_qna'] = out_executives.apply(lambda x: \
                              len(x['exec_mention_qna']), axis=1)
out_executives['exec_count_transcript'] = out_executives.apply(lambda x: \
                              len(x['exec_mention_transcript']), axis=1)
out_executives['exec_count_total']= out_executives.exec_count_qna+out_executives.exec_count_transcript


html/1.html
html/10.html
html/100.html


In [ ]:
# out_analyst.drop(['text','analyst_present','analysts','executive'],axis=1)
out_executives.drop(['text','executive','execs'],axis=1)\
.to_csv('analyst_out_foringestion.csv',sep=',', encoding = 'utf-8')
# out_executives.select({[}'file','quarter_yer'])

In [34]:
ls

 Volume in drive Z is Data
 Volume Serial Number is 647E-DA71

 Directory of Z:\cse315\textparsing

09/19/2018  02:07 PM    <DIR>          .
09/19/2018  02:07 PM    <DIR>          ..
09/19/2018  01:03 PM    <DIR>          .git
09/19/2018  01:07 PM    <DIR>          .ipynb_checkpoints
09/19/2018  01:35 PM    <DIR>          html
09/19/2018  01:35 PM                 0 log_extract.csv
09/19/2018  01:02 PM                 0 log_ner.csv
09/19/2018  01:02 PM           356,612 Named Entity Recognition Pipeline.ipynb
09/19/2018  01:02 PM             9,337 output_0912.xlsx
09/19/2018  01:02 PM           269,314 Parse Conversation HTML.ipynb
09/19/2018  01:02 PM                58 README.md
09/19/2018  01:03 PM    <DIR>          sample_out
09/19/2018  01:02 PM         3,211,605 Text Parsing Pipeline Analyst.ipynb
09/19/2018  02:07 PM           549,355 Text Parsing Pipeline.ipynb
               8 File(s)      4,396,281 bytes
               6 Dir(s)  17,942,880,256 bytes free


In [19]:
import csv
fieldnames = ['index','fname', 'status']
output_folder = PureWindowsPath('C:/Users/T.Santos18/Documents/workspace/textparsing/exec')
input_folder = PureWindowsPath('C:/Users/T.Santos18/Documents/workspace/textparsing/html')
file_list = os.listdir(input_folder)
out_list =[]
# for i in range(len(file_list)):
print('FILE\tANALYST_Flag\tFIRM\tQuarter\tYear') 
print(len(file_list))
for i in range(len(file_list)):
    with open(input_folder / file_list[i]) as inf:
        if(re.search('html',file_list[i],flags=re.IGNORECASE)):
            out_list.append(file_list[i])
#             print(file_list[i])

FILE	ANALYST_Flag	FIRM	Quarter	Year
91979


In [20]:
with open(output_folder / 'log_extract.csv', 'w') as csvfile:
# with open(data_folder / file_list[i]) as inf:

    logwriter = csv.writer(csvfile,delimiter='\t',quotechar='"',quoting=csv.QUOTE_MINIMAL)
#     logwriter.writerow(fieldnames)

 
    for ii in range(51200,60000):
#     for ii in range(1,4):
        try:
            total_rows = len(file_list)
            row_start = ii*1
            row_end = (ii*1)+1
        #     print('from: ', row_start, ' to: ', row_end)
#             filename = 'exec/exec_out_foringestion_'+str(row_start)+'.csv'
            filename = 'exec_out_foringestion_'+str(row_start)+'.csv'

#             print('FILE\tANALYST_Flag\tFIRM\tQuarter\tYear') 
#             print(ii,'####')
            out_list =[]

            for i in range(row_start,row_end):
                with open(input_folder / file_list[i]) as inf:
#                 with open(collection + '/' + file_list[i]) as inf:
                    if(re.search('html',file_list[i],flags=re.IGNORECASE)):
                        out_list.append(file_list[i])

            out_pd = pd.DataFrame(out_list)
            out_pd.columns=['file']
            out_pd['text'] = out_pd.apply(lambda x: extract_html(input_folder / x['file']), axis=1)
#             out_pd['text'] = out_pd.apply(lambda x: extract_html('html/'+x['file']), axis=1)
            out_pd['quarter'] = out_pd.apply(lambda x: extract_quarter(x['text']), axis=1)
            out_pd['year'] = out_pd.apply(lambda x: extract_year(x['text']), axis=1)
            # out_pd['quarter_year'] = out_pd.apply(lambda x: extract_q(x['text']), axis=1)
            out_pd['executive_firm'] = out_pd.apply(lambda x: extract_firm(x['text']), axis=1)
            out_pd['ticker'] = out_pd.apply(lambda x: extract_ticker(x['text']), axis=1)

            out_pd['analysts'] = out_pd.apply(lambda x: extract_analysts(x['text']), axis=1)
            out_pd['execs'] = out_pd.apply(lambda x: extract_executives(x['text']), axis=1)

            s = out_pd.execs.apply(pd.Series)\
                          .stack()\
                          .reset_index(level=1, drop=True)\
                          .to_frame('executive')
            out_executives = out_pd.join(s)                      


            out_executives['exec_name'] = out_executives.apply(lambda x: extract_name(x['executive']), axis=1)
            out_executives['exec_title'] = out_executives.apply(lambda x: extract_title(x['executive']), axis=1)

            out_executives['executive_content'] = out_executives.apply(lambda x: \
                                                     extract_content(x['text'],x['exec_name']), axis=1)
            out_executives['executive_qanda'] = out_executives.apply(lambda x: \
                                                     extract_qanda(x['text'],x['exec_name']), axis=1)

            out_executives['exec_qna_count'] = out_executives.apply(lambda x: \
                                                     basic_wc(x['executive_qanda']), axis=1)

            out_executives['exec_content_count'] = out_executives.apply(lambda x: \
                                                     basic_wc(x['executive_content']), axis=1)
            out_executives['exec_total_count'] = out_executives.exec_qna_count + out_executives.exec_content_count

            out_executives=out_executives.reset_index(level=0, drop=True)
            # out_analyst
            s = out_executives.execs.apply(pd.Series)\
                          .stack()\
                          .reset_index(level=1, drop=True)\
                          .to_frame('mention_name')
            out_executives = out_executives.join(s)          
            out_executives['mention_titile'] = out_executives.apply(lambda x: extract_title(x['mention_name']), axis=1)
            out_executives['mention_name'] = out_executives.apply(lambda x: extract_name(x['mention_name']), axis=1)
            out_executives['exec_mention_qna'] = out_executives.apply(lambda x: extract_name_pair(x['mention_name'],x['executive_qanda']), axis=1)
            out_executives['exec_mention_transcript'] = out_executives.apply(lambda x: extract_name_pair(x['mention_name'],x['executive_content']), axis=1)
            out_executives['mention_name_count_qna'] = out_executives.apply(lambda x: \
                                          len(x['exec_mention_qna']), axis=1)
            out_executives['mention_name_count_transcript'] = out_executives.apply(lambda x: \
                                          len(x['exec_mention_transcript']), axis=1)
            out_executives['mention_name_count_total']= out_executives.mention_name_count_qna+out_executives.mention_name_count_transcript
#             print('here##')
            # out_analyst.drop(['text','analyst_present','analysts','executive'],axis=1)
            out_executives.drop(['text','executive','execs','exec_mention_qna',\
                                 'exec_mention_transcript'],axis=1)\
            .to_csv(output_folder / filename,sep=',', encoding = 'utf-8',header=False)

        #             .to_csv(filename,sep=',', encoding = 'utf-8',header=False)
            # out_executives.select({[}'file','quarter_yer'])    
#             print('success: ', ii, ' ###')
            logwriter.writerow([ii,out_executives.file.head(1).item(),'success'])
        except:
#             print('exception: ', ii, ' ###')
            logwriter.writerow([ii,out_executives.file.head(1).item(),'fail'])
            
            pass

C:\Users\T.Santos18\Documents\workspace\textparsing\html\56816.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\56817.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\56818.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\56819.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\5682.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\56820.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\56821.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\56823.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\56824.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\56825.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\56826.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\56828.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\56829.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\5683.html
C:\Users\T.Santos18\Documents\workspace\textparsin

C:\Users\T.Santos18\Documents\workspace\textparsing\html\56932.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\56933.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\56934.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\56935.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\56936.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\56937.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\56939.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\5694.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\56940.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\56941.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\56942.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\56943.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\56944.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\56945.html
C:\Users\T.Santos18\Documents\workspace\textparsi

C:\Users\T.Santos18\Documents\workspace\textparsing\html\57046.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57047.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57048.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57049.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\5705.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57050.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57051.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57052.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57053.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57054.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57055.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57056.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57057.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57058.html
C:\Users\T.Santos18\Documents\workspace\textparsi

C:\Users\T.Santos18\Documents\workspace\textparsing\html\57160.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57161.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57162.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57163.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57164.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57165.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57166.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57167.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57168.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57169.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\5717.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57170.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57171.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57172.html
C:\Users\T.Santos18\Documents\workspace\textparsi

C:\Users\T.Santos18\Documents\workspace\textparsing\html\57272.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57273.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57275.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57276.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57277.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57278.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57279.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\5728.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57280.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57281.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57283.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57284.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57285.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\57286.html
C:\Users\T.Santos18\Documents\workspace\textparsi

...

C:\Users\T.Santos18\Documents\workspace\textparsing\html\64236.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64237.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64238.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64239.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\6424.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64240.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64241.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64242.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64243.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64245.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64246.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64247.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64248.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64249.html
C:\Users\T.Santos18\Documents\workspace\textparsi

C:\Users\T.Santos18\Documents\workspace\textparsing\html\64355.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64356.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64357.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64358.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64359.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\6436.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64361.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64362.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64363.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64364.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64365.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64366.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64367.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64368.html
C:\Users\T.Santos18\Documents\workspace\textparsi

C:\Users\T.Santos18\Documents\workspace\textparsing\html\64472.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64473.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64474.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64475.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64476.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64477.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64478.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64479.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\6448.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64480.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64481.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64482.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64483.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64485.html
C:\Users\T.Santos18\Documents\workspace\textparsi

C:\Users\T.Santos18\Documents\workspace\textparsing\html\64598.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64599.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\646.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\6460.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64600.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64601.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64602.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64603.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64604.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64605.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64606.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64607.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64608.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64609.html
C:\Users\T.Santos18\Documents\workspace\textparsing

C:\Users\T.Santos18\Documents\workspace\textparsing\html\64718.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64719.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\6472.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64720.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64723.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64724.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64725.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64726.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64727.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64728.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64729.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\6473.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64730.html
C:\Users\T.Santos18\Documents\workspace\textparsing\html\64731.html
C:\Users\T.Santos18\Documents\workspace\textparsin

In [ ]:
import csv
fieldnames = ['index','fname', 'status']

with open('log_extract.csv', 'wb') as csvfile:
    writer = csv.DictWriter(csvfile,delimiter='\t',quotechar='"',quoting=csv.QUOTE_NONE, fieldnames=fieldnames)
    writer.writer(['test','test','success'])

In [40]:
out_analyst.file.head(1).item()


'1.html'